# Tokenization

In [ ]:
!pip install transformers

## Loading a Turkish Pre-trained Tokenizer

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizerTUR = AutoTokenizer.from_pretrained(
    "dbmdz/bert-base-turkish-uncased",
)
print(f"VOC size is: {tokenizerTUR.vocab_size}")
print(f"The model is {type(tokenizerTUR)}")

## Loading an English Pre-trained Tokenizer

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizerEN = AutoTokenizer.from_pretrained("bert-base-uncased")
print(f"VOC size is: {tokenizerEN.vocab_size}")
print(f"The model is {type(tokenizerEN)}")

In [ ]:
word_en = "telecommunications"
print(f"is in Turkish Model ? {word_en in tokenizerTUR.vocab}")
print(f"is in English Model ? {word_en in tokenizerEN.vocab}")

In [ ]:
tokens = tokenizerTUR.tokenize(word_en)
tokens

But, The pieces are in the Turkish model

In [ ]:
[t in tokenizerTUR.vocab for t in tokens]

In [ ]:
tokens = tokenizerEN.tokenize(word_en)
tokens

In [ ]:
long_word_tur = "Muvaffakiyetsizleştiricileştiriveremeyebileceklerimizdenmişsinizcesine"

'''
It means that “As though you happen to have been from among those whom we will not be able to easily/quickly make a maker of unsuccessful ones” 
'''

In [ ]:
print(tokenizerTUR.tokenize(long_word_tur))

## Understanding Tokenization Algorithms

### Train tokenizers from scratch

let's load Shakespeare plays from gutenberg project

In [ ]:
import nltk
from nltk.corpus import gutenberg

nltk.download("gutenberg")
nltk.download("punkt")
plays = ["shakespeare-macbeth.txt", "shakespeare-hamlet.txt", "shakespeare-caesar.txt"]
shakespeare = [" ".join(s) for ply in plays for s in gutenberg.sents(ply)]

In [ ]:
# We prepare a template for the post-processing
# Some initial settings

In [ ]:
from tokenizers.processors import TemplateProcessing

special_tokens = ["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
temp_proc = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", special_tokens.index("[CLS]")),
        ("[SEP]", special_tokens.index("[SEP]")),
    ],
)

## Training BPE

In [ ]:
from tokenizers import Tokenizer
from tokenizers.normalizers import Sequence, Lowercase, NFD, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.decoders import BPEDecoder

# Instantiate BPE (Byte-Pair Encoding)
tokenizer = Tokenizer(BPE())

# a unicode normalizer, lowercasing and , replacing accents in order  :
# * Sequence : It composes multiple PreTokenizer that will be run in the given order
tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])

# Whitespace: Splits on word boundaries using the regular expression \w+|[^\w\s]+
tokenizer.pre_tokenizer = Whitespace()
tokenizer.decoder = BPEDecoder()
tokenizer.post_processor = temp_proc

We are ready to train the model 

In [ ]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(vocab_size=5000, special_tokens=special_tokens)
tokenizer.train_from_iterator(shakespeare, trainer=trainer)
print(f"Trained vocab size: {tokenizer.get_vocab_size()}")

In [ ]:
# take a sentence from macbeth

In [ ]:
sen = "Is this a dagger which I see before me, the handle toward my hand?"
sen_enc = tokenizer.encode(sen)
print(f"Output: {format(sen_enc.tokens)}")

In [ ]:
sen_enc2 = tokenizer.encode("Macbeth and Hugging Face")

In [ ]:
print(f"Output: {format(sen_enc2.tokens)}")

In [ ]:
# Let us pass  two sentences

In [ ]:
two_enc = tokenizer.encode("I like Hugging Face!", "He likes Macbeth!")

In [ ]:
print(f"Output: {format(two_enc.tokens)}")

In [ ]:
tokenizer.model.save(".")

In [ ]:
!wc -l ./merges.txt

In [ ]:
!head -6 ./merges.txt

In [ ]:
!head -1000 ./merges.txt| tail -5

In [ ]:
# Save and Load Tokenizer

In [ ]:
tokenizer.save("MyBPETokenizer.json")
tokenizerFromFile = Tokenizer.from_file("MyBPETokenizer.json")
sen_enc3 = tokenizerFromFile.encode("I like HuggingFace and Macbeth")
print(f"Output: {format(sen_enc3.tokens)}")

## Training WordPiece

In [ ]:
from tokenizers.models import WordPiece
from tokenizers.decoders import WordPiece as WordPieceDecoder
from tokenizers.normalizers import BertNormalizer

# BERT normalizer includes cleaning the text, handling accents, chinese chars and lowercasing

tokenizer = Tokenizer(WordPiece())
tokenizer.normalizer = BertNormalizer()
tokenizer.pre_tokenizer = Whitespace()

tokenizer.decoder = WordPieceDecoder()

In [ ]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(
    vocab_size=5000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

tokenizer.train_from_iterator(shakespeare, trainer=trainer)
output = tokenizer.encode(sen)
print(output.tokens)

In [ ]:
# let us use WordPiece Decoder to treat the sentences properly.

In [ ]:
tokenizer.decode(output.ids)

In [ ]:
# force the model to produce UNK tokens

In [ ]:
tokenizer.encode("Kralsın aslansın Macbeth!").tokens

# Pre-made tokenizers 
* CharBPETokenizer: The original BPE
* ByteLevelBPETokenizer: The byte level version of the BPE
* SentencePieceBPETokenizer: A BPE implementation compatible with the one used by SentencePiece
* BertWordPieceTokenizer: The famous Bert tokenizer, using WordPiece

In [ ]:
# Fast Tokenizers optimized for Research and Production

In [ ]:
from tokenizers import (
    ByteLevelBPETokenizer,
    CharBPETokenizer,
    SentencePieceBPETokenizer,
    BertWordPieceTokenizer,
)

In [ ]:
tokenizer = SentencePieceBPETokenizer()
print(tokenizer.normalizer)
print(tokenizer.pre_tokenizer)
print(tokenizer.decoder)
print(tokenizer.post_processor)

In [ ]:
tokenizer = BertWordPieceTokenizer()
print(tokenizer.normalizer)
print(tokenizer.pre_tokenizer)
print(tokenizer.decoder)
print(tokenizer.post_processor)